In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('train.csv')
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1000009418151094273,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,10000169349117863715,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,10000371904215119486,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,10000640724480838376,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,10000679056417042096,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [ ]:
df['device_id']

0       a99f214a
1       a99f214a
2       a99f214a
3       a99f214a
4       a99f214a
          ...   
2495    a99f214a
2496    a99f214a
2497    a99f214a
2498    a99f214a
2499    a99f214a
Name: device_id, Length: 2500, dtype: object

In [ ]:
len(df['device_id'].unique())

293

In [ ]:
def contagem(x, bin_column, label):
  label_1 = pd.Series(x[x[label] > 0][bin_column].value_counts(), name=label)
  label_0 = pd.Series(x[x[label] < 1][bin_column].value_counts(), name='no_'+label)

  counts = pd.DataFrame([label_1, label_0]).T.fillna('0')
  counts['total_' + label] = counts[label].astype('int64') + counts['no_'+label].astype('int64')
  return counts

def bin_counting(counts, label):
  counts['Bin+'] = counts[label].astype('int64').divide(counts['total_'+label].astype('int64'))
  counts['Bin-'] = counts['no_' + label].astype('int64').divide(counts['total_'+label].astype('int64'))

  counts['Probs'] = (counts['Bin+']).divide(counts['Bin-'] + 1)

  bin_counts = counts.filter(items=['Bin+', 'Bin-', 'Probs'])
  return counts, bin_counts

In [ ]:
count = contagem(df, 'device_id', 'click')
count, bin_counts = bin_counting(count, 'click')

bin_counts

,Bin+,Bin-,Probs
0.093233,0.170564,0.829436,0.093233
1.000000,1.000000,0.000000,1.000000
0.076923,0.142857,0.857143,0.076923
0.333333,0.500000,0.500000,0.333333
0.000000,0.000000,1.000000,0.000000


In [ ]:
df['device_id'] = df['device_id'].map(lambda x: bin_counts['Probs'][x] if x in df['device_id'].values else np.NaN)
df['device_id']

0       0.093233
1       0.093233
2       0.093233
3       0.093233
4       0.093233
          ...   
2495    0.093233
2496    0.093233
2497    0.093233
2498    0.093233
2499    0.093233
Name: device_id, Length: 2500, dtype: float64